In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.2 pyspark-shell'

In [2]:
from pyspark.sql.session import SparkSession
import json

In [3]:
spark = SparkSession \
        .builder \
        .appName("test").getOrCreate()

In [4]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:19092") \
    .option("subscribe", "hello") \
    .option("includeHeaders", "true") \
    .option("startingOffsets", "earliest") \
    .load()

In [9]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- headers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: string (nullable = true)
 |    |    |-- value: binary (nullable = true)



In [10]:
df.createOrReplaceTempView("updates")
spark.sql("select count(*) from updates")

DataFrame[count(1): bigint]

In [12]:
df.select("value")

DataFrame[value: binary]